## Importing necessary libraries

In [ ]:
import keras
from keras.preprocessing import image
from keras.layers import Conv2D, UpSampling2D, InputLayer, Input, Reshape, merge, MaxPooling2D
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

## Mounting Google Drive to import our data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/new data'
train_datagen = ImageDataGenerator(rescale = 1/255)

## Image Pre-processing

In [ ]:
train = train_datagen.flow_from_directory(path, target_size=(256, 256),class_mode=None,batch_size=305) 

Found 341 images belonging to 4 classes.


### Normalization of image

In [ ]:
X =[]
Y =[]
for img in train[0]:
  try:
      lab = rgb2lab(img)
      X.append(lab[:,:,0])
      Y.append(lab[:,:,1:] / 128)
  except:
     print('error')
X = np.array(X)
Y = np.array(Y)
X = X.reshape(X.shape+(1,))
print(X.shape)
print(Y.shape)

(305, 256, 256, 1)
(305, 256, 256, 2)


## Model

In [ ]:
import tensorflow.keras.layers as layers  
model = keras.Sequential([
    #Encoding the data
    layers.Input(shape=(256,256,1)),
    layers.Conv2D(64, (3,3), activation='relu', padding='same', strides=2),
    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.Conv2D(128, (3,3), activation='relu', padding='same', strides=2),
    layers.Conv2D(256, (3,3), activation='relu', padding='same'),
    layers.Conv2D(256, (3,3), activation='relu', padding='same', strides=2),
    layers.Conv2D(512, (3,3), activation='relu', padding='same'),
    layers.Conv2D(512, (3,3), activation='relu', padding='same'),
    layers.Conv2D(256, (3,3), activation='relu', padding='same'),

    #Decoding the data
    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.UpSampling2D((2, 2)),
    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.UpSampling2D((2, 2)),
    layers.Conv2D(32, (3,3), activation='relu', padding='same'),
    layers.Conv2D(16, (3,3), activation='relu', padding='same'),
    layers.Conv2D(2, (3, 3), activation='tanh', padding='same'),
    layers.UpSampling2D((2, 2))
    ])

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_52 (Conv2D)           (None, 128, 128, 64)      640       
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 64, 64, 128)       147584    
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 64, 64, 256)       295168    
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 32, 32, 256)       590080    
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 32, 32, 512)       1180160   
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 32, 32, 512)      

### Fitting the model

In [ ]:
model.compile(optimizer='adamax', loss='mse' , metrics=['accuracy'])
model.fit(X,Y,validation_split=0.2, epochs=500)

Epoch 1/500
8/8 [==============================] - 5s 658ms/step - loss: 0.5431 - accuracy: 0.4539 - val_loss: 0.0253 - val_accuracy: 0.6257
Epoch 2/500
8/8 [==============================] - 2s 296ms/step - loss: 0.0277 - accuracy: 0.6280 - val_loss: 0.0229 - val_accuracy: 0.6336
Epoch 3/500
8/8 [==============================] - 2s 294ms/step - loss: 0.0261 - accuracy: 0.6362 - val_loss: 0.0212 - val_accuracy: 0.6593
Epoch 4/500
8/8 [==============================] - 2s 297ms/step - loss: 0.0239 - accuracy: 0.6801 - val_loss: 0.0265 - val_accuracy: 0.6537
Epoch 5/500
8/8 [==============================] - 2s 297ms/step - loss: 0.0249 - accuracy: 0.6864 - val_loss: 0.0199 - val_accuracy: 0.6602
Epoch 6/500
8/8 [==============================] - 2s 297ms/step - loss: 0.0219 - accuracy: 0.6980 - val_loss: 0.0180 - val_accuracy: 0.7387
Epoch 7/500
8/8 [==============================] - 2s 298ms/step - loss: 0.0207 - accuracy: 0.7484 - val_loss: 0.0162 - val_accuracy: 0.7419
Epoch 8/500
8

## Coloring the images

In [ ]:
test_path = '/content/drive/MyDrive/new data/test/'
test = os.listdir(test_path)
print (test)
colorizer = []
for imgName in test:
  img = img_to_array(load_img('/content/drive/MyDrive/new data/test/'+imgName)) # importing the test data
  img = resize(img ,(256,256)) # resizing the image to 256*256
  colorizer.append(img)
colorizer = np.array(colorizer, dtype = 'uint8')
colorizer = rgb2lab(1.0/255*colorizer)[:,:,:,0] # storing the grayscale part in colorizer
colorizer = colorizer.reshape(colorizer.shape+(1,))
print(colorizer.shape)

['00000138_(6).jpg', '00000140.jpg', '00000146_(4).jpg', '00000146_(6).jpg', '00000144_(5).jpg', '00000180.jpg', '00000191_(6).jpg', '00000192_(3).jpg', '00000192_(5).jpg', '00000193_(4).jpg', '00000195.jpg', '00000196_(3).jpg', '00000199_(6).jpg', '00000129.jpg', '00000142.jpg', '00000161.jpg', '00000199.jpg', '00000241.jpg']
(18, 256, 256, 1)


In [ ]:
output = model.predict(colorizer) # predicting the colors i.e. A and  B channels
output = output * 128

In [ ]:
# Coloring the images
for i in range(len(output)):
    result = np.zeros((256, 256, 3)) # initializing empty array
    result[:,:,0] = colorizer[i][:,:,0] # storing grayscale part in result's zeroth channel
    result[:,:,1:] = output[i] # storing A and B color channel in 1 and 2 channel respectively
    imsave("/content/drive/MyDrive/new data/results/"+str(i)+".png", lab2rgb(result)) # converting to RGB and saving the image